# Simplex algorithm

## [Michel Bierlaire](https://people.epfl.ch/michel.bierlaire), EPFL.

In [1]:
import numpy as np

In this notebook, we present a possible implementation of the simplex algorithm, and we apply it on four examples. 

This is Algorithm 16.2 in <a href="http://optimizationprinciplesalgorithms.com/">Bierlaire (2015) Optimization: principles and algorithms, EPFL Press.</a>

The simplex algorithm solves a linear optimization problem written in standard form: \\[ \min_{x \in \mathbb{R}^n} c^T x \\] subject to \\[ \begin{aligned} Ax & = b, \\ x & \geq 0, \end{aligned}\\] where $A \in \mathbb{R}^{m \times n}$, $b \in \mathbb{R}^m$, and $c \in \mathbb{R}^n$.

In [2]:
def simplex(A, b, c, basis):
    """
    :param A: m x n matrix
    :type A: numpy.array 2D
    
    :param b: m vector
    :type b: numpy.array 1D
    
    :param c: n vector
    :type c: numpy.array 1D
    
    :param basis: list of indices of the variables in the initial basis
    :type basis: list(int)
    """
    # Dimension of the problem
    m, n = A.shape
    if b.shape[0] != m:
        raise Exception(f'Incompatible sizes: A is {m}x{n}, b is of length {b.shape[0]}, and should be {m}')
    if c.shape[0] != n:
        raise Exception(f'Incompatible sizes: A is {m}x{n}, c is of length {c.shape[0]}, and should be {n}')

    # Initialization
    optvalue = np.inf
    iters = list()
    stop = False
    while not stop:
        # Extract the basis matrix
        B = A[:, basis]
       
        # Calculate the value of the basic variables
        xb = np.linalg.solve(B,b)

        # Each column of the matrix d is the dB vector of the corresponding index
        minusd = np.linalg.solve(B,A)
        # Calculation of the cost, for reporting only
        cost = c[basis].T @ xb
        # Calculation of the reduced costs
        reducedCost = c.T - c[basis].T @ minusd
        # Identify the negative reduced costs
        negativeReducedCost = reducedCost < 0
        if not negativeReducedCost.any():
            iters.append([xb, cost, basis.copy(), reducedCost.copy(), None, None, None, None])
            optimalbasis = basis
            unbounded = False
            stop = True
        else:
            # In Python, True is larger than False. The next statement returns the 
            # index of a True entry in the array, that is the index of a negative reduced cost.
            # It is the index of the variable that will enter the basis.
            p = np.argmax(negativeReducedCost)
            

            # Calculate the maximum step that can be done along the basic direction d[p]
            steps = np.array([xb[k] / minusd[k][p] if minusd[k][p] > 0 else np.inf for k in range(m)])
            q = np.argmin(steps)            
            step = steps[q]
            
            # Store the quantities calculated during this iteration
            iters.append([xb, cost, basis.copy(), reducedCost, -minusd[:, p], step, p, q])

            if step == np.inf:
                # The problem is unbounded
                optimalbasis = np.zeros((m,1))
                unbounded = True
                stop = True
            else:
                # Variable q is replaced by variable p in the basis.
                basis[q] = p
    return optimalbasis, unbounded, iters

The following function is designed to report the information of a given iteration

In [3]:
def oneIteration(iteration):
    xb = iteration[0]
    cost = iteration[1]
    basicIndices = iteration[2]
    reducedCosts = iteration[3]
    basicDirection = iteration[4]
    step = iteration[5]
    p = iteration[6]
    q = iteration[7]
    
    n = len(reducedCosts)
    strBasis = ', '.join(f'{k+1:2}' for k in basicIndices) 
    nonbasicIndices = list(set(range(n)) - set(basicIndices))
    strReducedCosts = ', '.join(f'{reducedCosts[k]:+6.2f}' for k in nonbasicIndices)
    x = [0] * n
    for i, ik in enumerate(basicIndices):
        x[ik] = xb[i]
    strx = ', '.join(f'{k:+6.2f}' for k in x)
    strdir = ', '.join(f'{k:+6.2f}' for k in basicDirection) if basicDirection is not None else ''
    if basicDirection is None:
        return f'{strBasis} | {strReducedCosts} | {strx} | {cost:+6.2f}'
    else:        
        return f'{strBasis} | {strReducedCosts} | {strx} | {cost:+6.2f} | {strdir} | {step:4.2f} | {p+1:2} | {q+1:2}'

# Example 1

Consider the optimization problem in standard from with \\[A = \left(\begin{array}{cccc}  1 & 1 & 1 & 0 \\ 1 & -1 & 0 & 1 \end{array} \right), \qquad b= \left(\begin{array}{c}  1 \\ 1 \end{array} \right), \qquad c= \left(\begin{array}{c} -1 \\ -2 \\0\\0 \end{array} \right). \\]

In [4]:
A = np.array([[1, 1, 1, 0], [1, -1, 0, 1]])
b = np.array([1, 1])
c = np.array([-1, -2, 0, 0])
basis = [2, 3]
optimalbasis, unbounded, iters = simplex(A, b, c, basis)
for i in iters:
    print(oneIteration(i))
print('\n[Legend: Basis | Reduced costs | x | cost | dB | alpha | p | q]')

 3,  4 |  -1.00,  -2.00 |  +0.00,  +0.00,  +1.00,  +1.00 |  +0.00 |  -1.00,  -1.00 | 1.00 |  1 |  1
 1,  4 |  -1.00,  +1.00 |  +1.00,  +0.00,  +0.00,  +0.00 |  -1.00 |  -1.00,  +2.00 | 1.00 |  2 |  1
 2,  4 |  +1.00,  +2.00 |  +0.00,  +1.00,  +0.00,  +2.00 |  -2.00

[Legend: Basis | Reduced costs | x | cost | dB | alpha | p | q]


The optimal solution is \\[x^*=\left(\begin{array}{c} 0\\1 \\ 0 \\2 \end{array}\right) \\] with optimal cost -2.

# Example 2

Consider the optimization problem in standard from with \\[A = \left(\begin{array}{rrrrr}  1 & 3 & 0 & 4 & 1 \\ 1 & 2 & 0 & -3 & 1 \\ -1 & -4 & 3 & 0 & 0 \end{array} \right), \qquad b= \left(\begin{array}{c}  2 \\ 2 \\ 1\end{array} \right), \qquad c= \left(\begin{array}{r} 2 \\ 3 \\ 3 \\ 1 \\ -2 \end{array} \right). \\]

In [5]:
A = np.array([[1, 3, 0, 4, 1], [1, 2, 0, -3, 1], [-1, -4, 3, 0, 0]])
b = np.array([2, 2, 1])
c = np.array([2, 3, 3, 1, -2])
basis = [0, 1, 2]
optimalbasis, unbounded, iters = simplex(A, b, c, basis)
for i in iters:
    print(oneIteration(i))
print('\n[Legend: Basis | Reduced costs | x | cost | dB | alpha | p | q]')

 1,  2,  3 |  +3.00,  -5.00 |  +2.00,  -0.00,  +1.00,  +0.00,  +0.00 |  +7.00 |  -1.00,  +0.00,  -0.33 | 2.00 |  5 |  1
 5,  2,  3 |  +5.00, -82.00 |  +0.00,  -0.00,  +0.33,  +0.00,  +2.00 |  -3.00 | +17.00,  -7.00,  -9.33 | -0.00 |  4 |  2
 5,  4,  3 |  +5.00, +11.71 |  +0.00,  +0.00,  +0.33,  -0.00,  +2.00 |  -3.00

[Legend: Basis | Reduced costs | x | cost | dB | alpha | p | q]


The optimal solution is \\[x^*=\left(\begin{array}{c} 0\\ 0 \\ \frac{1}{3} \\ 0 \\ 2 \end{array}\right) \\] with optimal cost -3.

# Example 3

Consider the optimization problem in standard from with \\[A = \left(\begin{array}{rrrrrr}  1& 2& 2& 1& 0& 0 \\ 2& 1& 2& 0& 1& 0 \\ 2& 2& 1& 0& 0& 1 \end{array} \right), \qquad b= \left(\begin{array}{c}  20\\ 20\\ 20\end{array} \right), \qquad c= \left(\begin{array}{r} -10\\ -12\\ -12\\ 0\\ 0\\ 0 \end{array} \right). \\]

In [6]:
A = np.array([[1, 2, 2, 1, 0, 0], [2, 1, 2, 0, 1, 0], [2, 2, 1, 0, 0, 1]])
b = np.array([20, 20, 20])
c = np.array([-10, -12, -12, 0, 0, 0])
basis = [3, 4, 5]
optimalbasis, unbounded, iters = simplex(A, b, c, basis)
for i in iters:
    print(oneIteration(i))
print('\n[Legend: Basis | Reduced costs | x | cost | dB | alpha | p | q]')

 4,  5,  6 | -10.00, -12.00, -12.00 |  +0.00,  +0.00,  +0.00, +20.00, +20.00, +20.00 |  +0.00 |  -1.00,  -2.00,  -2.00 | 10.00 |  1 |  2
 4,  1,  6 |  -7.00,  -2.00,  +5.00 | +10.00,  +0.00,  +0.00, +10.00,  +0.00,  +0.00 | -100.00 |  -1.50,  -0.50,  -1.00 | 0.00 |  2 |  3
 4,  1,  2 |  -9.00,  -2.00,  +7.00 | +10.00,  +0.00,  +0.00, +10.00,  +0.00,  +0.00 | -100.00 |  -2.50,  -1.50,  +1.00 | 4.00 |  3 |  1
 3,  1,  2 |  +3.60,  +1.60,  +1.60 |  +4.00,  +4.00,  +4.00,  +0.00,  +0.00,  +0.00 | -136.00

[Legend: Basis | Reduced costs | x | cost | dB | alpha | p | q]


The optimal solution is \\[x^*=\left(\begin{array}{c} 4 \\ 4 \\ 4 \\ 0 \\ 0 \\ 0\end{array}\right) \\] with optimal cost -136.

# Example 4

Consider the following linear optimization problem: \\[ \min_{x\in\mathbf{R}^2} -3 x_1 - 2 x_2
\\]
subject to
\\[
\begin{aligned}
  x_1 - x_2 & \geq -2, \\
  2x_1 + x_2 & \leq 8, \\
  x_1 + x_2 & \leq 5, \\ 
  x_1 + 2x_2 & \leq 10,\\
  x_1, x_2 & \geq 0.
\end{aligned}
\\]

In standard form, we have\\[A = \left(\begin{array}{rrrrrr}  -1 & 1 & 1 & 0 & 0 & 0 \\
2 & 1 & 0 & 1 & 0 & 0 \\
1 & 1 & 0 & 0 & 1 & 0 \\
1 & 2 & 0 & 0 & 0 & 1 \end{array} \right), \qquad b= \left(\begin{array}{c}  2 \\ 8 \\ 5  \\ 10\end{array} \right), \qquad c= \left(\begin{array}{c} -3 \\ -2 \\ 0
  \\ 0 \\ 0 \\ 0 \end{array} \right). \\]

In [7]:
A = np.array([[-1, 1, 1, 0, 0, 0], [2, 1, 0, 1, 0, 0], [1, 1, 0, 0, 1, 0], [1, 2, 0, 0, 0, 1]])
b = np.array([2, 8, 5, 10])
c = np.array([-3, -2, 0, 0, 0, 0])
basis = [2, 3, 4, 5]
optimalbasis, unbounded, iters = simplex(A,b,c,basis)
for i in iters:
    print(oneIteration(i))
print('\n[Legend: Basis | Reduced costs | x | cost | dB | alpha | p | q]')

 3,  4,  5,  6 |  -3.00,  -2.00 |  +0.00,  +0.00,  +2.00,  +8.00,  +5.00, +10.00 |  +0.00 |  +1.00,  -2.00,  -1.00,  -1.00 | 4.00 |  1 |  2
 3,  1,  5,  6 |  -0.50,  +1.50 |  +4.00,  +0.00,  +6.00,  +0.00,  +1.00,  +6.00 | -12.00 |  -1.50,  -0.50,  -0.50,  -1.50 | 2.00 |  2 |  3
 3,  1,  2,  6 |  +1.00,  +1.00 |  +3.00,  +2.00,  +3.00,  +0.00,  +0.00,  +3.00 | -13.00

[Legend: Basis | Reduced costs | x | cost | dB | alpha | p | q]


The optimal solution is \\[x^*=\left(\begin{array}{c} 3 \\ 2 \\ 3 \\ 0 \\ 0 \\ 3\end{array}\right) \\] with optimal cost -13.